In [47]:
import os
import pandas as pd
import zipfile
import csv
from io import StringIO
import mysql.connector
import re

### Testing on home server

In [48]:
try:
    connection = mysql.connector.connect(
        host = '192.168.1.26',
        user = 'aportra',
        password = '@$Occer22',
        database = 'wedge_assignment')

    cursor = connection.cursor()
except mysql.connector.Error as err:
    print(f'{err}')

finally:
    if connection.is_connected():
        connection.close()
        print('Connection Closed')
    
    if cursor:
        print('Cursor created')

Connection Closed
Cursor created


#### Setting Directory for Mac and Windows 

In [49]:
try:
    directory_small = '/Users/aaronportra/Documents/ADA/wedge_project/WedgeZipOfZips_small/'
    directory = '/Users/aaronportra/Documents/ADA/wedge_project/WedgeZipOfZips/'
    os.listdir(directory)
except FileNotFoundError:
    directory_small = 'C:\\Users\\aport\\OneDrive\\Documents\\School\\Fall Semester 2024\\Applied Data Analytics\\wedge_project\\WedgeZipOfZips_small\\'
    directory = 'C:\\Users\\aport\\OneDrive\\Documents\\School\\Fall Semester 2024\\Applied Data Analytics\\wedge_project\\WedgeZipOfZips\\'
    os.listdir(directory)

## Cleaning Data for BQ

In [50]:
def sniff(file):

    file = StringIO(file)

    sample = file.read(10000)

    sniffer = csv.Sniffer()

    file.seek(0)

    delimiter = sniffer.sniff(sample).delimiter


    reader = csv.reader(file, delimiter=delimiter)

    first_row = next(reader)

    has_header = sniffer.has_header(sample)


    if len(first_row) < 50:
        has_header = False

    return delimiter, has_header

In [51]:
# transactions = {}

# for file in os.listdir(directory):
#     if file.endswith('.zip'):
#         with zipfile.ZipFile(os.path.join(directory, file),'r') as zip_file:
#             for info in zip_file.namelist():
    
#                 file_content = zip_file.read(info).decode('utf-8',errors = 'replace')

#                 csv_file = StringIO(file_content)

#                 h = sniff(file_content)

#                 csv_file.seek(0)

#                 if h[1]:
#                     data = pd.read_csv(csv_file, delimiter=h[0])
#                     data = pd.DataFrame(data)

#                     columns = data.columns


#                 elif not h[1]:
#                     data = pd.read_csv(csv_file, delimiter = h[0], header = None)
#                     data = pd.DataFrame(data)
#                     data.columns = columns
 

#             transactions[file.rstrip('.zip')] = data   

In [66]:
#Removing Null values and replacing with white space
def del_nan(data):
    
    data = data.fillna(None)

#Adding space to text. Example: ChickenBreast -> Chicken Breast
def add_space(text):
    if isinstance(text,str):
        return re.sub(r'([a-z])([A-Z])',r'\1 \2',text)

    return text


In [53]:
print(small_zips['transArchive_201001_201003_small']['cost'].convert_dtypes())

0       2.648
1        4.37
2       0.938
3        3.12
4        3.12
        ...  
9995      0.0
9996      0.0
9997      6.3
9998     2.35
9999     2.65
Name: cost, Length: 10000, dtype: Float64


In [71]:
def create_db(directory):
    transactions = {}

    for file in os.listdir(directory):
        if file.endswith('.zip'):
            with zipfile.ZipFile(os.path.join(directory, file),'r') as zip_file:
                for info in zip_file.namelist():
        
                    file_content = zip_file.read(info).decode('utf-8',errors = 'replace')

                    csv_file = StringIO(file_content)

                    h = sniff(file_content)

                    csv_file.seek(0)

                    if h[1]:
                        data = pd.read_csv(csv_file, delimiter=h[0])
                        data = pd.DataFrame(data)

                        columns = data.columns


                    elif not h[1]:
                        data = pd.read_csv(csv_file, delimiter = h[0], header = None,)
                        data = pd.DataFrame(data)
                        data.columns = columns
    
                    del_nan(data)

                    data = data.convert_dtypes()

                transactions[file.rstrip('.zip')] = data


 

    for trans in transactions:
        for column in transactions[trans]:
            if transactions[trans][column].dtypes == 'O':
                transactions[trans][column] = transactions[trans][column].apply(add_space)


    
    return transactions

In [72]:
small_zips = create_db(directory_small)

# full_zips = create_db(directory)

In [58]:
# save_path = '/Users/aaronportra/Documents/Advanced Data Analytics/wedge_project/clean_transactions/'

# for trans in full_zips:
#     df = transactions[trans].to_csv(save_path + trans + '.csv', encoding = 'utf-8', index = False)

In [81]:
small_zips['transArchive_201001_201003_small'].to_csv('clean_data.csv',index = False)